In [1]:
import pandas as pd
import numpy as np
import json
from haversine import haversine
import datetime
import os
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

print(datetime.datetime.now())
print(os.getcwd())

2020-05-10 23:13:43.151682
/home/jian/Projects/Big_Lots/Analysis/2020_Q2/review_others_work


In [2]:
df_DMA_zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,usecols=[0,2],skiprows=1)
df_DMA_zip.columns=['zip_cd','DMA']
df_DMA_zip=df_DMA_zip.drop_duplicates()


In [3]:
df_store_list=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200501-152115-660.txt",
                         dtype=str,sep="|")
df_store_list['zip_cd']=df_store_list['zip_cd'].apply(lambda x: x.split("-")[0].split(" ")[0].zfill(5))
df_store_list=df_store_list[['location_id','zip_cd','latitude_meas','longitude_meas']]
df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)
df_store_list=df_store_list[~df_store_list['location_id'].isin(['145','6990'])]

In [4]:
file_ta=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
df_ta_by_store=file_ta.parse("view_by_store", dtype=str)

In [5]:
df_zip_by_store=pd.DataFrame()

for ind, row in df_ta_by_store.iterrows():
    location_id = row['location_id']
    P_zips = eval(row['trans_P_zips_70_within_TA'])
    S_zips = eval(row['trans_S_zips_70_within_TA'])
    zip_10 = eval(row['zips_in_10'])



    df_P = pd.DataFrame(list(zip([location_id] * len(P_zips), P_zips)))
    if len(df_P) > 0:
        df_P.columns = ['location_id', 'zip_cd']
        df_P['zip_type'] = "P"



    df_S = pd.DataFrame(list(zip([location_id] * len(S_zips), S_zips)))
    if len(df_S) > 0:
        df_S.columns = ['location_id', 'zip_cd']
        df_S['zip_type'] = "S"



    df_10 = pd.DataFrame(list(zip([location_id] * len(zip_10), zip_10)))
    if len(df_10) > 0:
        df_10.columns = ['location_id', 'zip_cd']
        df_10['zip_type'] = "zip_10"



    df_zip_by_store = df_zip_by_store.append(df_P).append(df_S).append(df_10)



In [6]:
df_store_list_temp=df_store_list[['location_id','latitude_meas','longitude_meas']]
df_zip_by_store=pd.merge(df_store_list_temp,df_zip_by_store,on="location_id",how="left")
df_zip_by_store_existing=df_zip_by_store[pd.notnull(df_zip_by_store['zip_cd'])]
df_zip_by_store_new=df_zip_by_store[pd.isnull(df_zip_by_store['zip_cd'])]
df_zip_by_store_new.loc[df_zip_by_store_new['location_id']=="5416","latitude_meas"]=38.9093141
df_zip_by_store_new.loc[df_zip_by_store_new['location_id']=="5416","longitude_meas"]=-78.1833013

print(df_zip_by_store_new.shape)
df_zip_by_store_new.head(2)

(36, 5)


/home/jian/.local/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,location_id,latitude_meas,longitude_meas,zip_cd,zip_type
15294,4679,42.723314,-84.424187,NaN,NaN
15604,4702,31.789767,-106.405255,NaN,NaN


In [7]:
del df_zip_by_store_new['zip_cd']
del df_zip_by_store_new['zip_type']

df_zip_by_store_new=df_zip_by_store_new.reset_index()
del df_zip_by_store_new['index']
df_new_zips=pd.DataFrame()
for i, row in df_zip_by_store_new.iterrows():
    store_loc=[row['latitude_meas'],row['longitude_meas']]
    location_id=row['location_id']
    list_zip_for_new_store=[]
    for zip_cd, v in zip_centers.items():
        dist=haversine(store_loc,v,unit="mi")
        if dist<=10:
            list_zip_for_new_store.append(zip_cd)
    df=pd.DataFrame({"location_id":[location_id]*len(list_zip_for_new_store),"zip_cd":list_zip_for_new_store},index=[0]*len(list_zip_for_new_store))
    df_new_zips=df_new_zips.append(df)
    

In [8]:
df_zip_by_store_new=pd.merge(df_zip_by_store_new,df_new_zips,on="location_id")

In [9]:
del df_zip_by_store_existing['zip_type']
df_ta_zip_by_store=df_zip_by_store_existing.append(df_zip_by_store_new).drop_duplicates()
df_ta_zip_by_store.head(2)

,location_id,latitude_meas,longitude_meas,zip_cd
0,3,33.42157,-82.018921,30906
1,3,33.42157,-82.018921,30904


In [10]:
df_ta_zip_by_store.shape,df_ta_zip_by_store['location_id'].nunique()

((23311, 4), 1404)

In [11]:
df_store_type=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/YG_zips_all_current_stores/Store List Report 04.23.20.xlsx",dtype=str)
df_store_type=df_store_type[['Store','SOTF']]
df_store_type=df_store_type[df_store_type['SOTF'].str.lower()=="yes"]

In [12]:
df_ta_zip_by_store['Store_Type']=np.where(df_ta_zip_by_store['location_id'].isin(df_store_type['Store'].tolist()),"SOTF","Legacy")
df_ta_zip_by_store['Store_Type'].nunique()

2

In [13]:
df_ta_zip_by_store=df_ta_zip_by_store[['location_id','zip_cd','Store_Type']].rename(columns={"zip_cd":"ta_zip"})


In [14]:
df_store_list=df_store_list.rename(columns={"zip_cd":"store_zip"})
df_ta_zip_by_store=pd.merge(df_ta_zip_by_store,df_store_list,on="location_id",how="left")

In [15]:
df_output_detail=df_DMA_zip.rename(columns={"zip_cd":"store_zip"})
df_output_detail=pd.merge(df_output_detail,df_ta_zip_by_store,on="store_zip",how="left")

In [16]:
df_1_DMA=df_output_detail.groupby("DMA")['store_zip'].nunique().to_frame().reset_index()

In [17]:
df_2_stores=df_output_detail[pd.notnull(df_output_detail['location_id'])]
df_2_stores=df_2_stores.groupby(["DMA","Store_Type"])['ta_zip','location_id'].nunique().reset_index()
df_2_stores.head(2)

,DMA,Store_Type,ta_zip,location_id
0,ABILENE-SWEETWATER,Legacy,35,2
1,"ALBANY, GA",Legacy,19,1


In [18]:
df_2_stores=df_2_stores.pivot(index="DMA",columns="Store_Type",values=["ta_zip","location_id"]).reset_index()
df_2_stores.columns=['DMA','Legacy_zip_counts','SOTF_zip_counts','Legacy_store_counts','SOTF_store_counts']
df_2_stores.head(2)

,DMA,Legacy_zip_counts,SOTF_zip_counts,Legacy_store_counts,SOTF_store_counts
0,ABILENE-SWEETWATER,35.0,NaN,2.0,NaN
1,"ALBANY, GA",19.0,10.0,1.0,1.0


In [19]:
df_output=pd.merge(df_1_DMA,df_2_stores,on="DMA",how="outer")
df_output=df_output.fillna(0)
df_output['total_store']=df_output['Legacy_store_counts']+df_output['SOTF_store_counts']
df_output=df_output[df_output['total_store']>0]

In [20]:
df_output

,DMA,store_zip,Legacy_zip_counts,SOTF_zip_counts,Legacy_store_counts,SOTF_store_counts,total_store
0,ABILENE-SWEETWATER,101,35.0,0.0,2.0,0.0,2.0
1,"ALBANY, GA",86,19.0,10.0,1.0,1.0,2.0
2,ALBANY-SCHENECTADY-TROY,390,104.0,0.0,8.0,0.0,8.0
3,ALBUQUERQUE-SANTA FE,398,63.0,0.0,9.0,0.0,9.0
5,ALPENA,18,10.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...
205,WILMINGTON,67,58.0,0.0,4.0,0.0,4.0
206,YAKIMA-PASCO-RCHLND-KNNWCK,80,18.0,0.0,4.0,0.0,4.0
207,YOUNGSTOWN,118,39.0,40.0,4.0,3.0,7.0
208,YUMA-EL CENTRO,31,8.0,0.0,1.0,0.0,1.0


In [22]:
for col in df_output.columns.tolist()[1:]:
    print(col,df_output[col].sum())

store_zip 40371
Legacy_zip_counts 9722.0
SOTF_zip_counts 5615.0
Legacy_store_counts 947.0
SOTF_store_counts 505.0
total_store 1452.0


In [65]:
df_output['total_store'].sum()

1452.0

In [67]:
pd.options.display.max_rows=200
df_output[df_output['total_store']>0]

,DMA,store_zip,Legacy_zip_counts,SOTF_zip_counts,Legacy_store_counts,SOTF_store_counts,total_store
0,ABILENE-SWEETWATER,101,35.0,0.0,2.0,0.0,2.0
1,"ALBANY, GA",86,19.0,10.0,1.0,1.0,2.0
2,ALBANY-SCHENECTADY-TROY,390,104.0,0.0,8.0,0.0,8.0
3,ALBUQUERQUE-SANTA FE,398,63.0,0.0,9.0,0.0,9.0
5,ALPENA,18,10.0,0.0,1.0,0.0,1.0
6,AMARILLO,162,16.0,0.0,3.0,0.0,3.0
8,ATLANTA,476,210.0,53.0,29.0,3.0,32.0
9,AUGUSTA-AIKEN,120,33.0,0.0,4.0,0.0,4.0
10,AUSTIN,190,0.0,86.0,0.0,8.0,8.0
11,BAKERSFIELD,53,9.0,0.0,3.0,0.0,3.0


In [69]:
df_output_detail[df_output_detail['DMA']=="ALBANY, GA"]['location_id'].unique().tolist()

[nan, '5406', '5148']

In [60]:
a.shape

(173, 6)